In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

In [0]:
# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [0]:
# since the latest version of spacy have only 94 dims vectors so we will
# use a slightly advanced model having more vectors

In [18]:
spacy.__version__

'2.1.8'

In [6]:
# !python -m spacy download en_core_web_md

     |████████████████████████████████| 95.4MB 1.1MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126237 sha256=c9d462cd3a11c323bf22f28defdd95c4acf6ac03dbc571f19554199486391c70
  Stored in directory: /tmp/pip-ephem-wheel-cache-26fuo_3f/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
import en_core_web_md

In [0]:
dir_path = '/content/drive/My Drive/Colab Notebooks/AppliedAI/Quora_Assignment/Quora/'

In [0]:
# avoid decoding problems
df = pd.read_csv(dir_path+"train.csv")
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [22]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

In [0]:
qPair_train, qPair_test = train_test_split(questions,test_size=0.30,random_state =1234) 

In [0]:
tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [31]:
# en_vectors_web_md, having 300 vectors.
nlp = en_core_web_md.load()
# nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1)
    # 300 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    # print(doc1[0], doc1[0].vector.shape)
    for i,word1 in enumerate(doc1):
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1[i] += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)


100%|██████████| 404290/404290 [1:18:33<00:00, 85.77it/s]


In [38]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
    for i,word2 in enumerate(doc2):
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2[i] += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|██████████| 404290/404290 [1:19:12<00:00, 85.07it/s]


In [0]:
dir_path+"nlp_features_train.csv"

'/content/drive/My Drive/Colab Notebooks/AppliedAI/Quora_Assignment/Quora/nlp_features_train.csv'

In [0]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile(dir_path+'nlp_features_train.csv'):
    dfnlp = pd.read_csv(dir_path+"nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile(dir_path+'df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv(dir_path+"df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [0]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [0]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,66,66,54,54,0.166667
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,36,36,35,40,0.039216
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,46,56,0.175000


In [0]:
# data before preprocessing 
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [0]:
df3.head()

,id,q1_feats_m,q2_feats_m
0,0,"[14.07532426516215, -9.645537257194519, -4.587...","[11.636040467482347, -9.770243690564083, -2.42..."
1,1,"[9.6083123366038, -7.600832277039687, -7.44777...","[7.601154750585556, -2.2477694768458605, -5.17..."
2,2,"[5.450526501735052, -9.478967134157816, 0.0373...","[0.4482018134810708, -2.639052846214988, -10.7..."
3,3,"[-9.742455509992746, -4.595935399715717, -5.21...","[-0.579327384630839, -3.7459775706132254, -1.4..."
4,4,"[18.71525276079774, -11.789500057697296, -1.43...","[12.02181550860405, -8.95174354314804, 2.69811..."


In [0]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
0,14.075324,-9.645537,-4.587416,-10.244143,-5.995440,0.154087,9.116250,3.363274,-4.276731,3.775168,4.676592,-1.311206,0.214696,-7.258386,4.104015,-5.588730,2.908101,7.641409,1.290790,7.783487,-7.393653,-6.977053,4.718463,17.735287,3.863955,6.273638,12.548476,-3.193708,2.676813,20.253403,3.536485,-0.067393,-11.072747,-2.054275,1.686027,1.533048,-4.555358,5.574758,-7.384927,2.573798,...,3.781730,-0.307283,-2.649226,13.125337,-3.285175,-11.271361,11.973065,-3.967892,0.621331,4.707443,-4.153407,6.381485,-6.204066,12.314634,2.641095,-3.534450,5.250322,6.908222,8.590220,13.391931,0.627913,4.859366,-10.387130,-8.329206,-2.958715,-6.625498,-12.645274,6.419132,0.264400,-3.243253,2.211773,-6.832467,9.639716,3.374284,-1.425729,2.425707,-4.733321,0.276098,-8.758635,4.792305
1,9.608312,-7.600832,-7.447780,-6.992336,-5.882565,1.096122,3.882534,6.806343,0.401798,11.899900,-1.466905,0.719460,-12.529488,-8.792410,3.565104,-8.058200,7.807974,13.933509,6.983052,5.467685,-4.491069,-10.152142,1.896539,13.114872,-2.736560,-1.838878,8.648813,-4.657979,4.042351,18.207084,8.588940,3.112681,0.879714,-2.377999,10.699127,5.936232,-7.515456,6.729700,5.040957,-4.709711,...,4.735256,5.586630,-13.412940,-2.395623,-8.022929,3.690036,8.848420,0.725561,-2.378053,5.338559,4.759331,-8.931583,-0.248754,-3.117598,0.610729,8.641925,4.817085,9.430171,0.145096,-0.587143,4.687230,-1.261227,0.263725,-7.279807,-9.034836,-6.948301,-15.870230,3.222146,4.252779,1.953097,4.492419,1.475454,2.680720,0.016936,6.573346,-2.429068,-8.240649,4.785889,-1.024974,-0.700264
2,5.450527,-9.478967,0.037324,-6.977339,-5.610442,1.501007,7.701444,3.362464,-7.449395,3.447554,-3.367493,0.201992,-6.773672,-8.302512,8.637811,-2.342831,4.461372,7.390103,1.282980,9.395584,-9.094136,-8.757764,12.054949,17.732445,0.504918,3.536945,2.160323,-2.685914,0.465482,11.877291,2.997594,-1.876379,-9.002179,-4.196309,7.015830,6.598298,-0.150686,5.998041,-0.832973,2.759183,...,3.651254,-1.373355,-4.782258,1.959093,-0.062534,-6.682079,5.886943,-5.242220,-2.067467,0.201466,-1.099006,6.155071,-7.431587,12.393174,-0.604590,-3.933195,1.952880,10.471930,6.695909,5.731606,-1.365960,-0.648523,-3.383044,-9.346621,3.730729,-0.169901,-9.422568,5.083867,-3.752539,1.996983,4.115457,-3.321500,7.138706,-0.762095,-2.711183,-1.784786,-0.832138,0.117646,-6.688697,9.870747
3,-9.742456,-4.595935,-5.212554,-10.624210,-7.772208,6.780656,-1.762482,6.610879,2.137249,3.908512,-4.929629,-0.153111,0.238722,2.397743,-5.624527,4.479538,-6.981163,1.944471,-3.911522,-4.380379,-12.122809,-3.915134,11.276367,7.937025,2.636530,6.580427,5.251650,4.755544,-5.831067,9.365914,-0.270608,0.851892,-8.491198,7.190979,4.804028,5.227913,6.252564,4.296290,-2.707344,-4.893010,...,3.556050,0.787919,0.488868,8.629324,-1.953118,8.692670,-2.572887,0.554343,5.881808,2.244927,6.023340,-3.129079,-4.639558,-6.395542,-0.539373,-2.891152,-10.906829,2.679157,2.315490,-2.328044,-5.732536,-7.865270,11.091602,12.586215,-0.611709,0.418948,-4.887695,1.368204,-5.853313,-1.615859,9.065189,-1.003423,-0.126397,6.012185,1.826779,0.995843,-0.677484,9.489238,4.927465,-1.918715
4,18.715253,-11.789500,-1.434143,-17.105210,-11.780025,6.695190,10.934144,-4.502646,-6.143158,8.589998,-4.522437,2.726111,-4.609580,-5.829822,7.135352,-9.903358,1.804127,7.386136,-0.288650,17.421509,-10.878890,-13.769714,8.207163,22.324921,9.943438,14.127577,7.136395,-3.042985,2.549766,24.565204,0.443778,-5.053524,-10.205891,5.439491,11.845510,5.916786,-5.665240,8.369289,-0.318952,0.683242,...,2.634573,2.127200,2.064110,11.441577,-10.924169,-6.959953,18.731405,5.376158,1.187859,12.528015,1.356716,3.802125,-3.310510,13.958996,10.526731,-8.446784,6.917121,9.631176,13.795276,1.093943,-3.445324,-1.621175,-0.049359,-11.836444,-7.975289,-7.304109,-6.601110,15.945012,-7.438106,-6.70

In [0]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
0,11.636040,-9.770244,-2.426637,-10.992754,-7.480700,0.729674,8.206328,2.827246,-2.810916,4.089400,4.446060,-2.656859,1.509714,-5.794205,5.627758,-5.841816,2.368890,6.896128,0.573601,8.054659,-6.865797,-7.111285,5.508463,18.248683,3.085802,4.940985,12.769578,-3.197497,3.494162,21.496035,1.529243,1.153399,-8.333690,-0.737389,1.126353,1.674174,-4.033126,5.079078,-6.103480,1.516069,...,4.881463,-0.485503,-1.673882,14.979807,-3.674129,-11.124349,11.843755,-3.149620,2.888358,3.203674,-5.655329,5.400693,-5.797541,12.026896,3.400662,-1.811369,5.375621,7.485452,9.691081,15.160850,-0.162083,5.850073,-9.453534,-6.529911,-2.767179,-6.348161,-12.329663,4.222588,-0.787113,-3.903408,2.212569,-8.331279,10.085818,3.271610,-1.132787,1.212589,-4.594581,-0.735645,-7.677888,4.434125
1,7.601155,-2.247769,-5.177962,-6.423380,-5.928381,2.228896,6.895307,1.349237,-3.916418,4.328844,-1.915606,-4.102771,-8.446076,-6.807429,0.691428,-3.395997,8.023970,11.119089,-2.749201,-2.141884,-2.964589,-8.162599,5.741250,7.920323,-2.389248,-3.596312,5.621861,-5.757859,4.272665,20.170448,7.528309,3.859774,1.568598,1.639340,12.885109,4.693191,-7.812260,6.675930,7.850179,-6.419064,...,5.634361,-0.779097,-13.218461,1.681388,-7.759306,6.441947,13.147889,1.243449,-1.173433,3.096764,1.514099,-4.856911,-0.336880,2.110093,0.572735,-0.361317,-2.799929,9.838104,2.820943,-1.207417,-1.385374,-2.219010,0.797249,-6.218358,-3.126729,-11.247524,-13.395161,4.864974,-2.768333,0.275867,5.879403,0.261992,2.831124,4.602705,7.053165,-1.523310,-1.270949,4.356004,-1.179725,0.396846
2,0.448202,-2.639053,-10.709892,-8.939298,-1.733100,-0.897073,12.891655,8.297233,4.611564,1.165168,2.068638,-5.554158,-8.811517,-9.747402,3.871702,0.398902,4.853321,6.252268,-2.876084,7.762589,-12.853920,-2.454334,10.394323,18.936131,-4.067352,2.597949,10.852107,-6.975674,0.473101,14.357020,0.785941,2.835004,-15.463889,0.287613,5.444241,-0.847438,-1.687237,0.994497,4.560807,-0.399356,...,1.841008,-1.139386,-13.206790,12.102563,-3.263093,1.509210,4.663527,-10.141619,-8.520313,1.912440,3.324559,6.994229,-12.289523,10.584130,-1.615903,-3.327450,-0.890932,14.028286,2.580468,14.307397,-0.018446,-0.449445,3.903473,-3.236739,-5.459325,-8.460386,-14.837755,11.438155,2.702154,-4.804649,11.699996,1.584856,5.291213,8.058449,-7.723889,6.496577,-0.986718,-5.409244,-9.038666,8.666798
3,-0.579327,-3.745978,-1.445257,-5.120371,-0.622429,1.411834,7.920771,-0.219300,-1.087576,-2.336533,2.408482,-3.119963,-8.027774,-4.560816,-4.127363,3.605473,-0.724052,2.851221,-0.374831,2.330228,-4.569377,-0.406204,7.522588,3.916556,3.648294,-3.328880,2.905440,1.248225,-0.796385,14.190878,5.737325,4.997814,-3.563116,6.873636,-0.229901,1.743306,-0.466924,-2.202669,-3.622609,-1.309671,...,1.238356,-2.527213,-11.425533,2.781641,1.125819,-0.646599,3.414364,-4.348285,-0.761967,-1.202540,7.887838,7.048999,1.081569,1.871366,1.334438,-5.389807,-0.965825,1.837089,0.923682,6.639022,-0.225321,-3.524846,5.031487,-1.773816,2.349749,-3.459077,-10.450348,-0.415915,-2.386691,-4.485848,5.872938,-2.597856,7.173358,4.163979,2.145871,-2.301031,2.004233,-0.292013,-3.842984,4.686855
4,12.021816,-8.951744,2.698110,-11.592808,-13.330391,1.330849,11.523770,-5.070748,-4.342441,7.042565,-3.171151,-4.608253,4.934467,-0.212155,-0.678806,-3.047672,3.034721,6.838258,1.208911,9.685144,-10.105847,-13.538714,13.671781,15.459236,9.336110,7.335444,16.964040,-3.974513,-1.182304,13.724788,0.430663,-5.619239,-3.701312,2.337423,10.151105,4.001554,-6.754341,13.270675,-6.632818,3.194818,...,-0.007415,-0.289982,5.655502,10.775550,-13.007193,-9.958563,8.606956,-0.639778,-1.501038,6.260686,-9.170660,3.745033,-4.248764,4.755461,6.995585,-5.235469,3.516709,14.263176,10.076251,4.204021,-4.809246,-7.443386,-12.913243,-2.082745,-2.351839,-11.267435,-6.852753,16.36

In [0]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 96
Number of features in question2 w2v  dataframe : 96
Number of features in final dataframe  : 221


In [0]:
# storing the final features to csv file
save_path='/content/drive/My Drive/Colab Notebooks/AppliedAI/Quora_Assignment/'
if not os.path.isfile(save_path+'final_features_new.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv(save_path+'final_features_new.csv')

In [0]:
# os.path.isfile(dir_path+'final_features_new.csv')
dir_path+'final_features_new.csv'

'/content/drive/My Drive/Colab Notebooks/AppliedAI/Quora_Assignment/Quora/final_features_new.csv'

In [0]:
_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/AppliedAI/Quora_Assignment/final_features_new.csv',
                 nrows=100, index_col=0)